## Example ADVANCED mode recipe - normalization layer extra parameters tuning by SLaNC


In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import sys
import copy
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from evaluate import load
from dmx.compressor.modeling import DmxModel

def normalize(processor, text):
    try:
        res = processor.tokenizer.normalize(text)
    except:
        res = text.lower().strip()
    return res

def run_evaluation(pipe, dataset_list, processor,wer_metric,eval_name):
    """Helper function to run evaluation and return predictions/references"""
    predictions = []
    references = []
    
    print(f"Evaluating on {len(dataset_list)} samples...")
    
    for i, sample in enumerate(dataset_list):
        if i % 1 == 0:
            print(f"Processed {i}/{len(dataset_list)} samples")

        audio = sample["audio"]["array"]
        ground_truth = sample["text"]

        result = pipe(audio, return_timestamps=True)
        prediction = result["text"]

        predictions.append(normalize(processor, prediction))
        references.append(normalize(processor, ground_truth))
    wer_score = wer_metric.compute(predictions=predictions, references=references)
    print(f'***********{eval_name}\n prediction: {predictions} \n references: {references} \n wer: {wer_score}')
    return predictions, references , wer_score



1. Instantiate a `torch` model from source, HF hub in this case.

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
wer_metric = load("wer")
model_id = "openai/whisper-medium"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model = model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
task = "automatic-speech-recognition"

pipe = pipeline(
    task=task,
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device
)

dataset = load_dataset(
    "librispeech_asr", "clean", split="validation", streaming=True, trust_remote_code=True
)
dataset = dataset.take(2)
dataset_list = list(dataset)
predictions_gt, references_gt, wer_gt = run_evaluation(pipe, dataset_list, processor, wer_metric, 'vanilla')

Device set to use cuda:0
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Evaluating on 2 samples...
Processed 0/2 samples
Processed 1/2 samples
***********vanilla
 prediction: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 references: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 wer: 0.0


2. Transform into `DmxModel`; this does not change the functional behavior.

In [3]:
pipe.model = DmxModel.from_torch(pipe.model)

# -------------------------------------------------------------------------------
predictions_baseline, references_baseline, wer_baseline = run_evaluation(pipe, dataset_list, processor, wer_metric, 'baseline')

Evaluating on 2 samples...
Processed 0/2 samples


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Processed 1/2 samples
***********baseline
 prediction: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 references: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 wer: 0.0


3. Configure to BASIC mode; this should bring in all VSIMD approximations with default config.

In [4]:
pipe.model.to_basic_mode()

# -------------------------------------------------------------------------------
predictions_basic, references_basic, wer_basic = run_evaluation(pipe, dataset_list, processor, wer_metric, 'basic')

Evaluating on 2 samples...
Processed 0/2 samples
Processed 1/2 samples
***********basic
 prediction: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 references: ['he was in a fevered state of mind owing to the blight his wife is action threatened to cast upon his entire future', 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did'] 
 wer: 0.0


4. SLaNC calibrate `LayerNorm` instances.

In [5]:
from dmx.compressor import nn
from dmx.compressor.modeling import DmxModule
import re
from dmx.compressor.advanced_recipe import (
    DmxSLaNCHyperparams,
    DmxSLaNCRecipe,
)


def hp_gen(_model) -> dict:
    _hp = {}
    
    complete_gm = list(_model._gms.values())[0]
    named_dmx_modules = [(n,m) for (n,m) in complete_gm.named_modules() if isinstance(m, DmxModule)]

    for _n, _m in named_dmx_modules:
        if isinstance(_m, nn.LayerNorm):
            if ".layer_norm" in _n:
                # final layer norm
                layers = pipe.model.get_submodule(_n.replace(".layer_norm", ".layers", -1))
                layers = list(layers.children())
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    mlp_type="standard",
                    device=_m.weight.device,
                    prev_ln_weight=layers[-1].final_layer_norm,
                    fc1=layers[-1].fc1,
                    fc2=layers[-1].fc2
                )
            elif "self_attn_layer_norm" in _n and ".0." not in _n:
                layer_num = int(''.join(re.findall(r'\.\d+\.', _n)).replace(".", "", -1))
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    mlp_type="standard",
                    device=_m.weight.device,
                    prev_ln_weight=pipe.model.get_submodule(
                        _n.replace("self_attn_layer_norm", "final_layer_norm", -1)
                        .replace("." + str(layer_num) + ".", "." + str(layer_num - 1) + ".", -1)),
                    fc1=pipe.model.get_submodule(
                        _n.replace("self_attn_layer_norm", "fc1", -1)
                        .replace("." + str(layer_num) + ".", "." + str(layer_num - 1) + ".", -1)),
                    fc2=pipe.model.get_submodule(
                        _n.replace("self_attn_layer_norm", "fc2", -1)
                        .replace("." + str(layer_num) + ".", "." + str(layer_num - 1) + ".", -1))
                )
            elif "encoder_attn_layer_norm" in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_attn",
                    device=_m.weight.device,
                    prev_ln_weight=pipe.model.get_submodule(
                        _n.replace("encoder_attn_layer_norm", "self_attn_layer_norm", -1)),
                    v_proj=pipe.model.get_submodule(
                        _n.replace("encoder_attn_layer_norm", "self_attn.v_proj", -1)),
                    o_proj=pipe.model.get_submodule(
                        _n.replace("encoder_attn_layer_norm", "self_attn.out_proj", -1))
                )
            elif "final_layer_norm" in _n:
                if ".encoder." in _n:
                    prev_ln_weight = pipe.model.get_submodule(
                        _n.replace("final_layer_norm", "self_attn_layer_norm", -1)
                    )
                    v_proj = pipe.model.get_submodule(
                        _n.replace("final_layer_norm", "self_attn.v_proj", -1)
                    )
                    o_proj = pipe.model.get_submodule(
                        _n.replace("final_layer_norm", "self_attn.out_proj", -1)
                    )
                elif ".decoder." in _n:
                    prev_ln_weight = pipe.model.get_submodule(
                        _n.replace("final_layer_norm", "encoder_attn_layer_norm", -1)
                    )
                    v_proj=pipe.model.get_submodule(
                        _n.replace(".decoder.", ".encoder.", -1)
                        .replace("final_layer_norm", "self_attn.v_proj", -1)
                    )
                    o_proj=pipe.model.get_submodule(
                        _n.replace("final_layer_norm", "encoder_attn.out_proj", -1)
                    )
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_attn",
                    device=_m.weight.device,
                    prev_ln_weight=prev_ln_weight,
                    v_proj=v_proj,
                    o_proj=o_proj
                )
            elif "self_attn_layer_norm" in _n and ".0." in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="first",
                    device=_m.weight.device
                )
    return _hp

with DmxSLaNCRecipe(hp_gen).applied_to(pipe.model):
    print("SLaNC done!")

from dmx.compressor.modeling.model import DmxConfig

complete_gm = list(pipe.model._gms.values())[0]
all_modules_config  = DmxConfig({'_gm.'+n:m.dmx_config() for n,m in complete_gm.named_modules() if isinstance(m,DmxModule)})
pipe.model.configure(all_modules_config)

predictions_slanc,references_slanc,wer_slanc = run_evaluation(pipe,dataset_list,processor,wer_metric,'basic_slanc')

Norm is 32.99658203125
Norm is 25.339387893676758
Norm is 25.267427444458008
Norm is 24.136276245117188
Norm is 28.710100173950195
Norm is 29.268850326538086
Norm is 32.396549224853516
Norm is 29.621387481689453
Norm is 29.464366912841797
Norm is 30.218650817871094
Norm is 29.11838150024414
Norm is 32.09354782104492
Norm is 32.847068786621094
Norm is 34.551700592041016
Norm is 37.62411117553711
Norm is 37.165462493896484
Norm is 35.93863296508789
Norm is 42.9462890625
Norm is 37.44207000732422
Norm is 38.32508087158203
Norm is 46.133480072021484
Norm is 45.95112609863281
Norm is 39.404441833496094
Norm is 50.06367492675781
Norm is 35.80278015136719
Norm is 19.66062355041504
Norm is 11.302202224731445
Norm is 35.44587326049805
Norm is 8.609440803527832
Norm is 22.760784149169922
Norm is 11.468527793884277
Norm is 30.196537017822266
Norm is 19.79102325439453
Norm is 28.874479293823242
Norm is 31.28667449951172
Norm is 49.02684783935547
Norm is 24.505537033081055
Norm is 34.34408950805664

In [6]:
named_dmx_modules = [(n,m) for (n,m) in complete_gm.named_modules() if isinstance(m, DmxModule)]
for _n, _m in named_dmx_modules:
    if isinstance(_m, nn.LayerNorm):
        print(f"Name: {_n}, norm: {_m.approximator.function.extra_params}")


Name: model.encoder.layers.0.self_attn_layer_norm, norm: {'norm': 1.0}
Name: model.encoder.layers.0.final_layer_norm, norm: {'norm': tensor(0.0303, device='cuda:0')}
Name: model.encoder.layers.1.self_attn_layer_norm, norm: {'norm': tensor(0.0300, device='cuda:0')}
Name: model.encoder.layers.1.final_layer_norm, norm: {'norm': tensor(0.0395, device='cuda:0')}
Name: model.encoder.layers.2.self_attn_layer_norm, norm: {'norm': tensor(0.0313, device='cuda:0')}
Name: model.encoder.layers.2.final_layer_norm, norm: {'norm': tensor(0.0396, device='cuda:0')}
Name: model.encoder.layers.3.self_attn_layer_norm, norm: {'norm': tensor(0.0302, device='cuda:0')}
Name: model.encoder.layers.3.final_layer_norm, norm: {'norm': tensor(0.0414, device='cuda:0')}
Name: model.encoder.layers.4.self_attn_layer_norm, norm: {'norm': tensor(0.0315, device='cuda:0')}
Name: model.encoder.layers.4.final_layer_norm, norm: {'norm': tensor(0.0348, device='cuda:0')}
Name: model.encoder.layers.5.self_attn_layer_norm, norm: {